In [ ]:
# Coded By Sayan Chandra
# Roll : CS20M057
# DEEP LEARNING (CS6910) ASSIGNMENT 3
# Instructor : Mitesh M. Khapra  

In [ ]:
def unzip():
    !tar -xf dakshina_dataset_v1.0.tar

In [ ]:
def download():
    !wget https://storage.googleapis.com/gresearch/dakshina/dakshina_dataset_v1.0.tar

In [ ]:
def setWandb():
    global WANDB
    WANDB=0

In [ ]:
setWandb()
def download_unzip():
    download()
    unzip()
download_unzip()

In [ ]:
if WANDB:
   !pip install --upgrade wandb

In [ ]:
if WANDB: 
  !wandb login #952756aa88ee3a472980bceb7d23632ac0a85500

wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter: 
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


In [ ]:
import numpy as np
import tensorflow as tf

if WANDB:
  import wandb
  from wandb.keras import WandbCallback as WC
  
from tensorflow import keras

In [ ]:
sweep_config={
    'method' : 'bayes',
    'metric' : {
        'name' : 'val_accuracy(word_level)',
        'goal' : 'maximize',
    },
    'parameters' : {
        'input_embedding_size' : {
            'values' : [32, 64, 128],
        },
        'learning_rate' : {
            'values' : [1e-3, 2e-3, 3e-3],
        },
        'dropout' : {
            'values' : [0.2, 0.3, 0.5, 0.3]
        },
        'num_layers': { 
            'values' : [1, 2, 3]
        },
        'hidden_layer_size' : {
            'values' : [128, 512, 256],
        },
        'cell_type' : {
            'values' : [ "vanillaRnn", "lstm", "gru"],
        },
        'optimizer' : {
            'values' : ['rmsprop', 'adam', 'sgd'],
        },
        'epochs' : {
            'values' : [6, 17, 13],
        },
        'batch_size' : {
            'values' : [32, 128, 64],
        },
    }
}
if WANDB: sweep_id = wandb.sweep(sweep_config, entity="blackcloud", project="cs6910_dl_assignment_3")

In [ ]:
def eid_did_dtd(qwerty):
  did = np.zeros((len(qwerty), max_decoder_seq_length), dtype="float32") #2D  
  dtd = np.zeros((len(qwerty), max_decoder_seq_length, num_decoder_tokens), dtype="float32") #3D
  eid = np.zeros((len(qwerty), max_encoder_seq_length), dtype="float32") #2D
  return eid, did, dtd

In [ ]:
def bookKeep(qwerty):
    print("global vars are set for training the model.")
    print("Num_samples:", len(qwerty))
    print("# unique input tokens:", num_encoder_tokens) # unique chars in english
    print("# unique output tokens:", num_decoder_tokens) # unique chars in hindi
    print("Max sequence length for inputs:", max_encoder_seq_length) # max wordlen in english
    print("Max sequence length for outputs:", max_decoder_seq_length) # max wordlen in hindi

In [ ]:
# GLOBAL VARS
num_encoder_tokens=0; num_decoder_tokens=0; 
input_token_index={}; target_token_index={}
max_encoder_seq_length=0; max_decoder_seq_length=0;
input_characters=set()
target_characters = set()
##########################################################################################################################

def preprocessing_TrainValTest(dataType, datapath, mode, isTrain, en_format):
  print("preprocessing started for"+dataType)
  global max_encoder_seq_length, max_decoder_seq_length, input_token_index, target_token_index, num_encoder_tokens, num_decoder_tokens, input_characters, target_characters
  input_texts, target_texts = [], []
  with open(datapath, mode, encoding=en_format) as data:
    lines = data.read().split("\n")
  #print(len(lines))
  prev, __, _= lines[0].split("\t"); flag=1
  for iii, line in enumerate(lines[: len(lines) - 1]):
    target_text, input_text, ignore = line.split("\t")
    if iii>0 and target_text==prev : continue;
    prev=target_text
    #if target_text in target_texts : continue
    start = "\t"; end="\n"
    target_text = start + target_text + end ;  input_text = start + input_text + end
    input_texts.append(input_text) ;   target_texts.append(target_text)
    if isTrain:
      for cc in input_text:
          if cc not in input_characters:
              input_characters.add(cc)
      for cc in target_text:
          if cc not in target_characters:
              target_characters.add(cc)


  if isTrain==1:

    input_characters = sorted(list(input_characters))
    input_token_index = dict([(char, i) for i, char in enumerate(input_characters)])

    target_characters = sorted(list(target_characters))
    target_token_index = dict([(char, i) for i, char in enumerate(target_characters)])

    num_encoder_tokens = len(input_characters)
    max_encoder_seq_length = max([len(ii) for ii in input_texts])+1

    num_decoder_tokens = len(target_characters)
    max_decoder_seq_length = max([len(ii) for ii in target_texts])+1 

    bookKeep(input_texts)



  encoder_input_data, decoder_input_data, decoder_target_data= eid_did_dtd(input_texts)

  for i, (input_text, target_text) in enumerate(zip(input_texts, target_texts)):
      for t, char in enumerate(input_text):
          encoder_input_data[i, t] =  input_token_index[char]
      encoder_input_data[i, t + 1: ] = input_token_index[end]
      for t, char in enumerate(target_text):
          decoder_input_data[i, t] = target_token_index[char]
          if t > 0:
              decoder_target_data[i, t - 1, target_token_index[char]] = 1.0
      decoder_input_data[i, t + 1: ] = target_token_index[end]
      decoder_target_data[i, t:, target_token_index[end]] = 1.0
  print("preprocessing finished for"+dataType+"\n")
  return input_texts, target_texts, encoder_input_data, decoder_input_data, decoder_target_data

In [ ]:
def setPaths():
    aaaaaa="dakshina_dataset_v1.0/hi/lexicons/"
    bbbbbb =  aaaaaa+"hi.translit.sampled.train.tsv"
    cccccc =  aaaaaa+"hi.translit.sampled.dev.tsv"
    dddddd =  aaaaaa+"hi.translit.sampled.test.tsv"
    return aaaaaa, bbbbbb, cccccc, dddddd

In [ ]:
root, train_data_path, val_data_path, test_data_path = setPaths()
input_train_texts, target_train_texts, encoder_input_train_data, decoder_input_train_data, decoder_target_train_data = preprocessing_TrainValTest(" train", train_data_path, mode="r", isTrain=1, en_format="utf-8")
input_val_texts, target_val_texts, encoder_input_val_data, decoder_input_val_data, decoder_target_val_data = preprocessing_TrainValTest(" val" ,val_data_path, mode="r", isTrain=0, en_format="utf-8")
input_test_texts, target_test_texts, encoder_input_test_data, decoder_input_test_data, decoder_target_test_data = preprocessing_TrainValTest(" test" ,test_data_path, mode="r", isTrain=0, en_format="utf-8")

preprocessing started for train
global vars are set for training the model.
Num_samples: 25000
# unique input tokens: 28
# unique output tokens: 65
Max sequence length for inputs: 23
Max sequence length for outputs: 22
preprocessing finished for train

preprocessing started for val
preprocessing finished for val

preprocessing started for test
preprocessing finished for test



In [ ]:
def shapes():
   xx=keras.Input(shape=(max_encoder_seq_length,))
   yy=keras.Input(shape=(max_decoder_seq_length,))
   return xx, yy

def makeModel(xxx, yyy):
   return keras.Model(xxx, yyy)

In [ ]:
# RNN based seq2seq model

keepCells = {"gru":keras.layers.GRU, "lstm":keras.layers.LSTM, "vanillaRnn":keras.layers.SimpleRNN, "embedd":keras.layers.Embedding, "dense":keras.layers.Dense}

def ReccNeuralNet(en, input_embedd_sz, latent_dim, drop, cell):

    # (i) input layer for character embeddings
    global putEncoders, HiddenStatesOfDecoder, putDecoders

    encoder_inputs, decoder_inputs=shapes()

    encoder_layer=keepCells["embedd"](input_dim= num_encoder_tokens, output_dim=input_embedd_sz)
    encoder_embedded = encoder_layer(encoder_inputs)
    decoder_layer=keepCells["embedd"](input_dim = num_decoder_tokens, output_dim=input_embedd_sz )
    decoder_embedded=decoder_layer(decoder_inputs)
    # (i) input layer for character embeddings


    putEncoders, putDecoders, HiddenStatesOfDecoder, en = [], [], [], en-1
    de=en


    # (ii) one encoder RNN which sequentially encodes the input character sequence
    if cell=="vanillaRnn":

        ################################################################################################
        encoder = keepCells[cell](latent_dim , return_state=1, return_sequences=1)
        outputs, state_h = encoder(encoder_embedded)
        HiddenStatesOfDecoder.append([state_h])
        putEncoders.append(encoder)

        for i in range(en):
          en_rnn = keepCells[cell](latent_dim , return_state=1, return_sequences=1, dropout=drop)
          outputs, state_h = en_rnn(outputs)
          HiddenStatesOfDecoder.append([state_h])
          putEncoders.append(en_rnn)
        ################################################################################################

    elif cell=="lstm":

        ################################################################################################
        encoder = keepCells[cell](latent_dim , return_state=1, return_sequences=1)
        outputs, state_h, state_c = encoder(encoder_embedded)
        HiddenStatesOfDecoder.append([state_h, state_c])
        putEncoders.append(encoder)
        for i in range(en):
          en_lstm =keepCells[cell](latent_dim , return_state=1, return_sequences=1, dropout=drop)
          outputs, state_h, state_c = en_lstm(outputs)
          HiddenStatesOfDecoder.append([state_h, state_c])
          putEncoders.append(en_lstm)
        ################################################################################################

    elif cell=="gru":

        ################################################################################################
        encoder = keepCells[cell](latent_dim , return_state=1, return_sequences=1)
        outputs, state_h = encoder(encoder_embedded)
        HiddenStatesOfDecoder.append([state_h])
        putEncoders.append(encoder)
        for i in range(en):
          en_gru = keepCells[cell](latent_dim , return_state=1, return_sequences=1, dropout=drop)
          outputs, state_h = en_gru(outputs)
          HiddenStatesOfDecoder.append([state_h])
          putEncoders.append(en_gru)
        ################################################################################################


    # (ii) one encoder RNN which sequentially encodes the input character sequence
     



    # (iii) one decoder RNN which takes the last state of the encoder as input and produces one output character at a time

    if cell=="vanillaRnn":

        ################################################################################################
        decoder_rnn = keepCells[cell](latent_dim, return_sequences=1, return_state=1)
        de_outputs, ignore = decoder_rnn(decoder_embedded, initial_state=HiddenStatesOfDecoder[0])
        putDecoders.append(decoder_rnn)
        for i in range(de):
          de_rnn = keepCells[cell](latent_dim , return_state=1, return_sequences=1, dropout=drop)
          de_outputs, ignore = de_rnn(de_outputs,HiddenStatesOfDecoder[i+1])
          putDecoders.append(de_rnn)
        ################################################################################################

    elif cell=="lstm":

        ################################################################################################
        decoder_lstm = keepCells[cell](latent_dim, return_sequences=1, return_state=1)
        de_outputs, ignore1, ignore2 = decoder_lstm(decoder_embedded, initial_state=HiddenStatesOfDecoder[0])
        putDecoders.append(decoder_lstm)
        for i in range(de):
          de_lstm = keepCells[cell](latent_dim , return_state=1, return_sequences=1, dropout=drop)
          de_outputs, state_h, state_c = de_lstm(de_outputs, initial_state=HiddenStatesOfDecoder[i+1])
          putDecoders.append(de_lstm)
        ################################################################################################

    elif cell=="gru":

        ################################################################################################
        decoder_gru = keepCells[cell](latent_dim, return_sequences=1, return_state=1)
        de_outputs, ignore = decoder_gru(decoder_embedded, initial_state=HiddenStatesOfDecoder[0])
        putDecoders.append(decoder_gru)
        for i in range(de):
          de_gru = keepCells[cell](latent_dim , return_state=1, return_sequences=1, dropout=drop)
          de_outputs, ignore = de_gru(de_outputs, HiddenStatesOfDecoder[i+1])
          putDecoders.append(de_gru)
        ################################################################################################

    # (iii) one decoder RNN which takes the last state of the encoder as input and produces one output character at a time


  

    # Dense Layer
    makeDense=keepCells["dense"](num_decoder_tokens, activation="softmax")
    decoder_dense = makeDense
    decoder_outputs = decoder_dense(de_outputs) # None*22*65
    # Dense Layer
    
    #model = keras.Model([encoder_inputs, decoder_inputs], decoder_outputs)
    model = makeModel([encoder_inputs, decoder_inputs], decoder_outputs)
    return model


In [ ]:
def inferenceModel (mdl, ld, emb_var, attn, cell):
  print("Model has been trained. Making inference model now.")
  ld1,ld2=ld,ld
  input_to_encoder_embed = mdl.layers[0].output
  input_to_decoder_embed = mdl.layers[1].output
  en_embed_layer = mdl.layers[2]
  de_embed_layer = mdl.layers[3]
  en_embed_op = en_embed_layer(input_to_encoder_embed)
  de_embed_op= de_embed_layer(input_to_decoder_embed)
  statesEN = []
  outEN = en_embed_op  
  statesDEinps, statesDE = [], []

  
  for i in range(len(ld1)):
      if cell == 'lstm':
        encoder_lstm = putEncoders[i]
        #print(outEN.shape)
        outEN, state_h, state_c = encoder_lstm(outEN)
        statesEN += [state_h, state_c]
      else:
        #print("yo")
        encoder_gru_rnn = putEncoders[i]
        outEN, state_h = encoder_gru_rnn(outEN)
        statesEN += [state_h]

  inp1=input_to_encoder_embed
  out2=statesEN + [outEN]
  encoder_model = keras.Model(inputs=inp1, outputs = out2)



  for i in range(len(ld2)):
      if cell == 'lstm':
        statesCURinp = [keras.Input(shape=(ld2[i],)) for _ in range(2)]
        de_embed_op, state_h2, state_c2 = putDecoders[i](de_embed_op, initial_state=statesCURinp)
        statesDEinps += statesCURinp
        statesDE += [state_h2 , state_c2]

      else:
        #print("yo")
        statesCURinp = [keras.Input(shape=(ld2[i],)) for _ in range(1)]
        de_embed_op, state_h2=  putDecoders[i](de_embed_op, initial_state=statesCURinp)
        statesDEinps += statesCURinp
        statesDE += [state_h2 ]
        
  
  
  if attn == "no":
    print("This model is without attention.")
    #Dense layer
    decoder_dense = mdl.layers[4 + len(ld1) + len(ld2)]
    hiddenstatesDEinp = keras.Input(shape=(max_encoder_seq_length , ld1[0]))
    de_embed_op = decoder_dense(de_embed_op) 

    # Define the decoder model
    inp3=[input_to_decoder_embed]  + statesDEinps + [hiddenstatesDEinp]
    out2=[de_embed_op] + [statesDE]
    decoder_model = keras.Model(inp3, out2) 

    print("Inference model is made now.")
    return encoder_model , decoder_model


In [ ]:
def beam_search(mtrx, k):
    from math import log
    from numpy import array, argmax
    mtrx  = tf.nn.softmax(mtrx)
    sequences = [[list(), 0.0]]
    for eachRow in mtrx:
        family = list()
        for i in range(len(sequences)):
            seq, marks= sequences[i]
            best_k = np.argsort(eachRow)[-k:]
            for j in best_k:
                person = [seq + [j], marks+ tf.math.log(eachRow[j])]
                family.append(person)
        ordered = sorted(family, key=lambda tup:tup[1], reverse=True)
        sequences = ordered[:k]
    return np.array(sequences)[:,0:1]

In [ ]:
reverse_target_char_index = dict((i, char) for char, i in target_token_index.items())
def charByCharDecoding(input_seq):

    global beamMatrix
    beamMatrix=[] # matrix for keeping probabilities of each char of a particular word

    states_value = encoder_model.predict(input_seq) # predicting whole one-hot-encoded english word using encoder_model

    curChar = np.zeros((1, 1)); start="\t"
    curChar[0, 0] = target_token_index[start]
    # empty target seq curChar with 1st char index = start char index = idx("\t") = 0

    encoder_outputs=states_value[-1]
    encoder_states_value=states_value[:-1]
    # for decoder_model inputs

    halt = False
    while not halt:
        output_decoder, decoder_states = decoder_model.predict([curChar] + encoder_states_value + [encoder_outputs])
        # predicting each char using decoder_model sequentially

        beamMatrix.append(output_decoder[0][0])
        # append probabilities of char in beamMatrix

        sampled_token_index = np.argmax(output_decoder)
        # max prob

        if sampled_token_index == 1 :  halt = True
        # if char is "\n" i.e. INDEX = 1 then break cond. = True
        
        curChar = np.zeros((1, 1))
        curChar[0, 0] = sampled_token_index
        # empty target seq curChar with 1st char = current char index

        encoder_states_value = decoder_states
        # updation of prev with current
    return ""




In [ ]:
def WriteToCSVFile(arrays) :
  import csv
  f = open('/content/testPredictions.csv', 'w')
  for array in arrays:
      writer = csv.writer(f)
      writer.writerow(array)
  f.close()

In [ ]:
import numpy as np
def summary_architechture(model):
  print(model.summary())
  for i in range (len(model.layers)):
     print(i, model.layers[i])

In [ ]:
def plotModel(mmm):
  return tf.keras.utils.plot_model(mmm , show_shapes = True)

In [ ]:
class funcForWordAcc(tf.keras.callbacks.Callback):
  def __init__(self, strr, validation_data):
    self.strr=strr
    self.lsttt=[]
    self.asdfg = validation_data   
    
  def on_epoch_end(self, epoch, logs={}):
    
    ytrue = self.asdfg[1]
    ypred = self.model.predict(self.asdfg[0])

    count = 0
    one_hot_pred = tf.one_hot(tf.argmax(ypred, axis=2), ypred.shape[2]).numpy()
    for i in range(len(ypred)):
      count+=np.array_equal(ytrue[i], one_hot_pred[i])
    xxxxx=count/len(ypred)
    print(self.strr+"accword", xxxxx)
    if WANDB: wandb.log({self.strr+"_accuracy(word_level)" : xxxxx, "epoch":epoch})

In [ ]:
def TRAIN(model, opti, lr=1e-3, epk=15, bs=64):
    steps=1000
    from tensorflow.keras.experimental import CosineDecayRestarts as MyScheduler
    scheduler = (MyScheduler(lr, 1000))

    if opti=="adam":  optimizer = keras.optimizers.Adam(learning_rate=scheduler)
    elif opti=="sgd": optimizer = keras.optimizers.SGD(learning_rate=scheduler)
    else : optimizer = keras.optimizers.RMSprop(learning_rate=scheduler)

    model.compile(optimizer=optimizer, loss="categorical_crossentropy", metrics=["accuracy"])

    pcTrain = funcForWordAcc("train", validation_data = ([encoder_input_train_data, decoder_input_train_data], decoder_target_train_data))
    pcVal = funcForWordAcc("val", validation_data = ([encoder_input_val_data, decoder_input_val_data], decoder_target_val_data))
    
    history = model.fit(
        x=[encoder_input_train_data, decoder_input_train_data],
        y=decoder_target_train_data,
        shuffle=True,
        batch_size=bs,
        verbose=1,
        epochs=epk,
        validation_data= ([encoder_input_val_data, decoder_input_val_data], decoder_target_val_data),
        callbacks=([pcTrain, pcVal] if WANDB else [pcTrain, pcVal])
    )
    
    model.save("myrnnfinal")
    return model


In [ ]:
def WORDACCVAL(lengthOfPrediction=0, howMuch=0):
      n=len(encoder_input_val_data); 
      for seq_index in range(n):
          input_seq = encoder_input_val_data[seq_index : seq_index + 1]
          ignore = charByCharDecoding(input_seq).replace("\n","").strip()
          lengthOfPrediction=len(target_val_texts[seq_index].strip())
          result = beam_search(np.array(beamMatrix),3)
          target=decoder_input_val_data[seq_index,1:lengthOfPrediction+2]
          for seq in result:
            if np.array_equal(seq[0], target) :
              howMuch+=1; break
      print("val_inference_beam_acc", howMuch/n)
      return howMuch/n

In [ ]:
def Test_Acc_With_Best_Model_In_Sweep(lengthOfPrediction=0, howMuch=0):
      n=len(encoder_input_test_data);
      global arrays
      arrays=[['INPUT_WORD', 'PRED_WORD', 'TRUE_WORD']]
      print("calculation of Test inference accuracy has started...")
      for seq_index in range(n):
          flag=1
          input_seq = encoder_input_test_data[seq_index : seq_index + 1]
          ignore = charByCharDecoding(input_seq).replace("\n","").strip()
          lengthOfPrediction=len(target_test_texts[seq_index].strip())
          result = beam_search(np.array(beamMatrix),4)
          target=decoder_input_test_data[seq_index,1:lengthOfPrediction+2]
          for seq in result:
            if np.array_equal(seq[0], target) :
              ar=[]
              ar.append(input_test_texts[seq_index])
              ar.append("".join([reverse_target_char_index[gg] for gg in seq[0]]).strip())
              ar.append(target_test_texts[seq_index][1:-1])
              arrays.append(ar)
              howMuch+=1; flag=0; break
          if flag: 
              ar=[]
              ar.append(input_test_texts[seq_index])
              ar.append("".join([reverse_target_char_index[gg] for gg in result[0][0]]).strip())
              ar.append(target_test_texts[seq_index][1:-1])
              arrays.append(ar)              
          if (seq_index) and seq_index%499==0 : print("so far accuracy = ",howMuch/(seq_index+1))
      print("test_inference_beam_acc", howMuch/n)
      return howMuch/n

In [ ]:
#model = keras.models.load_model("myrnn")


def SweepParent():
      global num_layers, input_embedding_size, hidden_layer_size, dropout, cell_type, encoder_model, decoder_model
      if WANDB:
        start=wandb.init()
        config=start.config
        a,b,c,d,e=config.num_layers, config.input_embedding_size, config.hidden_layer_size ,config.dropout, config.cell_type,
        f,g,h,i=config.optimizer, config.learning_rate, config.epochs, config.batch_size
        num_layers, input_embedding_size, hidden_layer_size, dropout, cell_type=a,b,c,d,e
        wandb.run.name="~/cell_"+str(e)+"/drop_"+str(d)+"/opti_"+f+"/bs_"+str(i)+"/epoch_"+str(h)+"~"
        model=ReccNeuralNet(a,b,c,d,e)
        summary_architechture(model)
        model=TRAIN(model,f,g,h,i)
        encoder_model , decoder_model=inferenceModel(model, [hidden_layer_size for u in range(num_layers)],
                                                  [input_embedding_size, input_embedding_size], "no", cell_type) 
        valAcc=WORDACCVAL()
        wandb.log({"val_inference_beam_acc":valAcc})
      else:

        #{ best model from sweep runs ###
        a,b,c,d,e=2, 128, 256, 0.5, "lstm"
        f,g,h,i="rmsprop", 3e-3, 17, 32
        model=ReccNeuralNet(a,b,c,d,e)
        #} ##############################

        summary_architechture(model)
        num_layers, input_embedding_size, hidden_layer_size, dropout, cell_type=a,b,c,d,e
        model=TRAIN(model, f,g,h,i)
        encoder_model , decoder_model=inferenceModel(model, [hidden_layer_size for u in range(num_layers)], [input_embedding_size, input_embedding_size], "no", cell_type)  
      
      return model
      


In [ ]:
if WANDB: wandb.agent("ksbzk150", SweepParent)
else: model=SweepParent() # best test accuracy I got = 0.5596 (55.96%) 

In [ ]:
plotModel(model)

In [ ]:
ip,prd,actual=[],[],[]
for seq_index in [2444, 449, 5, 1021, 2014, 784, 997, 981, 1090, 73, 85, 1884, 0, 700, 600, 850]:
    input_seq = encoder_input_test_data[seq_index : seq_index + 1]
    #print(input_seq)
    ignore = charByCharDecoding(input_seq) 
    a=""
    lst=[np.argmax(iii) for iii in beamMatrix]
    reverse_target_char_index = dict((i, char) for char, i in target_token_index.items())
    for i in lst:
      a+=reverse_target_char_index[i]
    ip.append(input_test_texts[seq_index].strip()); 
    prd.append(a.replace("\n","").strip()); 
    actual.append(target_test_texts[seq_index].strip())
from termcolor import colored
for i in range(4):
    print(colored("i/p:  "+ip[i*4+0]+"          i/p: "+ ip[i*4+1]+ "          i/p: "+ ip[i*4+2]+ "          i/p: "+ ip[i*4+3],  'red', attrs=['bold']))
    print(colored("pred: "+prd[i*4+0]+"           pred: "+prd[i*4+1]+ "           pred: "+ prd[i*4+2]+ "           pred: "+prd[i*4+3], 'blue', attrs=['bold']))
    print(colored("true: " +actual[i*4+0]+"           true: "+actual[i*4+1]+ "           true: "+ actual[i*4+2]+ "           true: "+actual[i*4+3],  'green', attrs=['bold']))

    print("\n\n\n")



i/p:  hakikat          i/p: kritika          i/p: angraji          i/p: dinaank
pred: हकिकत           pred: कृतिका           pred: अंग्रजी           pred: दिनांक
true: हकीकत           true: कृतिका           true: अंग्रज़ी           true: दिनांक




i/p:  lenevali          i/p: junior          i/p: dastaanaa          i/p: thailee
pred: लेनेवाली           pred: जूनियर           pred: दस्ताना           pred: ठैली
true: लेनेवाली           true: जूनियर           true: दस्ताना           true: थैली




i/p:  naksalavaad          i/p: amarpaal          i/p: arabab          i/p: ramkot
pred: नकसलवाद           pred: अमरपाल           pred: अरबाब           pred: रामकोट
true: नक्सलवाद           true: अमरपाल           true: अरबाब           true: रामकोट




i/p:  ank          i/p: chhupata          i/p: ghumakkadi          i/p: toka
pred: अंक           pred: छुपता           pred: घुमककदी           pred: टोका
true: अंक           true: छुपता           true: घुमक्कड़ी           true: टोका






In [ ]:
def wannaTry(check, a=""):
    encodd= np.ones((1, max_encoder_seq_length), dtype="float32"); space="                 "

    for i, input_text in enumerate(["\t"+check+"\n"]):
      for t, char in enumerate(input_text):
          encodd[i, t] =  input_token_index[char]

    ignore=charByCharDecoding(encodd)
    lst=[np.argmax(iii) for iii in beamMatrix]
    reverse_target_char_index = dict((i, char) for char, i in target_token_index.items())
    print(check.strip(), space, "".join([reverse_target_char_index[g] for g in lst]).replace("\n","").strip())

checks=["sayan", "chandra", "shivam", "varun", "amrita", "aanchal", "mayank", "mitesh", "khapra", "rahul", "bharat", "rasta"]
print("input word:         predicted word:")
print("-----------         ---------------")
for check in checks:
  wannaTry(check)

input word:         predicted word:
-----------         ---------------
sayan                   सायन
chandra                   चंद्र
shivam                   शिवाम
varun                   वरुण
amrita                   अमृता
aanchal                   आंचल
mayank                   मायंक
mitesh                   मिटेश
khapra                   खपरा
rahul                   राहुल
bharat                   भारत
rasta                   रस्ता


In [ ]:
Test_Acc_With_Best_Model_In_Sweep()
WriteToCSVFile(arrays)